In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q https://downloads.apache.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
#!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!wget -q https://downloads.apache.org/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-preview2-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
!ls /usr/lib/jvm/

default-java		   java-11-openjdk-amd64     java-8-openjdk-amd64
java-1.11.0-openjdk-amd64  java-1.8.0-openjdk-amd64


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop2.7"

In [0]:

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:

import sys,tempfile, urllib
from pyspark.sql.functions import *

In [0]:
BASE_DIR = '/tmp'
CORONA_DATA_FILE = os.path.join(BASE_DIR, 'corona_data.csv')

In [0]:
corona_data=urllib.request.urlretrieve('https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/coronavirus/corona_dataset_latest.csv', CORONA_DATA_FILE)

In [0]:
BASE_DIR = '/tmp'
TWITTER_DATA_FILE = os.path.join(BASE_DIR, 'twitter_data.csv')

In [0]:
tweet_data=urllib.request.urlretrieve('https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/coronavirus/tweets.csv', TWITTER_DATA_FILE)

In [11]:
!ls /tmp


blockmgr-40d84bd2-f14b-41d1-bb8c-4e58cc665e7d
corona_data.csv
hsperfdata_root
spark-b83df1df-4d23-4902-a20e-bd7f7b89a35a
spark-c68a4004-3900-4f4c-84f3-de3ba7d74105
twitter_data.csv


In [0]:
corona_df = spark.read.option("inferSchema", "true").csv("/tmp/corona_data.csv", header=True)


In [13]:
corona_df.show()


+---+----------------+--------------------+--------+---------+----------+---------+-----+---------+--------------------+----+
|_c0|           State|             Country|     Lat|     Long|      Date|Confirmed|Death|Recovered|       state_cleaned|City|
+---+----------------+--------------------+--------+---------+----------+---------+-----+---------+--------------------+----+
|  0|            null|            Thailand|    15.0|    101.0|2020-01-22|        2|    0|        0|             Bangkok|null|
|  1|            null|               Japan|    36.0|    138.0|2020-01-22|        2|    0|        0|             Hiraide|null|
|  2|            null|           Singapore|  1.2833| 103.8333|2020-01-22|        0|    0|        0|           Singapore|null|
|  3|            null|               Nepal| 28.1667|    84.25|2020-01-22|        0|    0|        0|           Kathmandu|null|
|  4|            null|            Malaysia|     2.5|    112.5|2020-01-22|        0|    0|        0|             Sarawa

In [14]:
corona_df.count()

28143

In [0]:
twitter_df = spark.read.option("inferSchema", "true").csv("/tmp/twitter_data.csv", header=True)


In [16]:
twitter_df.count()


1000

In [17]:
twitter_df.show()


+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|_c0| geo|                text|                user|            location|            entities|           sentiment|             country|
+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|null|What is God sayin...|          petodinice|               Lagos|[('about #', 'CAR...|{'neg': 0.0, 'neu...|             Nigeria|
|  1|null|"BREAKING: ""this...| but i took the t...|                -… "|     JerryfranksonJF|      Abuja, Nigeria|"[(""Arsenal's Mi...|
|  2|null| #Coronavirus tes...|              cek422|   Pennsylvania, USA|                  []|{'neg': 0.173, 'n...|                 USA|
|  3|null| Get ready for ma...|        InfectiousDz|                 NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|                 USA|
|  4|null| The #coronavirus...|          

In [18]:
twitter_df.filter("country='USA'")

DataFrame[_c0: int, geo: string, text: string, user: string, location: string, entities: string, sentiment: string, country: string]

In [19]:
twitter_df.filter("country='USA'").show()


+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|  2|null| #Coronavirus tes...|         cek422|   Pennsylvania, USA|                  []|{'neg': 0.173, 'n...|    USA|
|  3|null| Get ready for ma...|   InfectiousDz|                 NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|    USA|
|  5|null| COVID-19 update ...| StewartNgilana|Durban | Port Eli...|[('Italy', 'GPE')...|{'neg': 0.178, 'n...|    USA|
|  6|null| It’s painful to ...|      BWheatnyc|             Florida|                  []|{'neg': 0.098, 'n...|    USA|
|  8|null| Questions about ...|    straightj23|        Columbus, OH|[('NAfME', 'CARDI...|{'neg': 0.0, 'neu...|    USA|
| 13|null| We’re the heck w...| harrytiffanyiv| 

In [20]:
twitter_df.filter("country='USA' and location like 'New%'").show()


+---+----+--------------------+---------------+------------------+--------------------+--------------------+-------+
|_c0| geo|                text|           user|          location|            entities|           sentiment|country|
+---+----+--------------------+---------------+------------------+--------------------+--------------------+-------+
| 31|null| I ordered Alex J...|       rcgillan|     New York, USA|[('Alex Jones', '...|{'neg': 0.109, 'n...|    USA|
| 49|null| This week we are...|  JamesWithers3|New York, New York|[('This week', 'D...|{'neg': 0.0, 'neu...|    USA|
|228|null|This is a very co...|baskingntheGlow|     New York City|[('hourly', 'TIME')]|{'neg': 0.12, 'ne...|    USA|
|238|null|I’m reposting thi...|   Veronicaromm|   New Jersey, USA|[('English', 'LAN...|{'neg': 0.0, 'neu...|    USA|
|261|null|Too early ...?  #...|      HJeppesen|      New York, NY|                ['']|{'neg': 0.0, 'neu...|    USA|
|275|null|  The Republican ...|  GenetBataiile|New Hampshire, US

In [0]:
tw_filter_df=twitter_df.filter("country='USA'")


In [22]:
tw_filter_df.explain()


== Physical Plan ==
*(1) Project [_c0#144, geo#145, text#146, user#147, location#148, entities#149, sentiment#150, country#151]
+- *(1) Filter (isnotnull(country#151) AND (country#151 = USA))
   +- BatchScan[_c0#144, geo#145, text#146, user#147, location#148, entities#149, sentiment#150, country#151] CSVScan Location: InMemoryFileIndex[file:/tmp/twitter_data.csv], ReadSchema: struct<_c0:int,geo:string,text:string,user:string,location:string,entities:string,sentiment:strin...




In [23]:
tw_filter_df.show(5)


+---+----+--------------------+--------------+--------------------+--------------------+--------------------+-------+
|_c0| geo|                text|          user|            location|            entities|           sentiment|country|
+---+----+--------------------+--------------+--------------------+--------------------+--------------------+-------+
|  2|null| #Coronavirus tes...|        cek422|   Pennsylvania, USA|                  []|{'neg': 0.173, 'n...|    USA|
|  3|null| Get ready for ma...|  InfectiousDz|                 NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|    USA|
|  5|null| COVID-19 update ...|StewartNgilana|Durban | Port Eli...|[('Italy', 'GPE')...|{'neg': 0.178, 'n...|    USA|
|  6|null| It’s painful to ...|     BWheatnyc|             Florida|                  []|{'neg': 0.098, 'n...|    USA|
|  8|null| Questions about ...|   straightj23|        Columbus, OH|[('NAfME', 'CARDI...|{'neg': 0.0, 'neu...|    USA|
+---+----+--------------------+--------------+----------

In [0]:
tw_filter_df=twitter_df.filter(col("location").startswith("N"))


In [25]:
tw_filter_df.explain()


== Physical Plan ==
*(1) Project [_c0#144, geo#145, text#146, user#147, location#148, entities#149, sentiment#150, country#151]
+- *(1) Filter (isnotnull(location#148) AND StartsWith(location#148, N))
   +- BatchScan[_c0#144, geo#145, text#146, user#147, location#148, entities#149, sentiment#150, country#151] CSVScan Location: InMemoryFileIndex[file:/tmp/twitter_data.csv], ReadSchema: struct<_c0:int,geo:string,text:string,user:string,location:string,entities:string,sentiment:strin...




In [26]:
tw_filter_df.show()


+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|      country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------------+
|  3|null| Get ready for ma...|   InfectiousDz|                 NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|          USA|
|  7|null| 📽️Friends, I wi...|        LorseaR|New South Wales, ...|[('Twitter', 'GPE...|{'neg': 0.123, 'n...|    Australia|
| 31|null| I ordered Alex J...|       rcgillan|       New York, USA|[('Alex Jones', '...|{'neg': 0.109, 'n...|          USA|
| 36|null| Before #coronavi...|   Frankapistan|       Nashville, TN|[('China', 'GPE')...|{'neg': 0.144, 'n...|          USA|
| 49|null| This week we are...|  JamesWithers3|  New York, New York|[('This week', 'D...|{'neg': 0.0, 'neu...|          USA|
|

In [27]:
twitter_df

DataFrame[_c0: int, geo: string, text: string, user: string, location: string, entities: string, sentiment: string, country: string]

In [28]:
twitter_df.first()

Row(_c0=0, geo=None, text='What is God saying to us about #coronavirus ?', user='petodinice', location='Lagos', entities="[('about #', 'CARDINAL')]", sentiment="{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'compound': 0.2732}", country='Nigeria')

In [29]:
twitter_df.take(5)


[Row(_c0=0, geo=None, text='What is God saying to us about #coronavirus ?', user='petodinice', location='Lagos', entities="[('about #', 'CARDINAL')]", sentiment="{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'compound': 0.2732}", country='Nigeria'),
 Row(_c0=1, geo=None, text='"BREAKING: ""this is disappointing', user=' but i took the test"". Arsenal\'s Mikel Arteta tests positive for #coronavirus', location='  -… "', entities='JerryfranksonJF', sentiment='Abuja, Nigeria', country='"[(""Arsenal\'s Mikel Arteta""'),
 Row(_c0=2, geo=None, text=' #Coronavirus testing must be made free to the public if we are going to understand the scope of this crisis. Anything le…', user='cek422', location='Pennsylvania, USA', entities='[]', sentiment="{'neg': 0.173, 'neu': 0.71, 'pos': 0.117, 'compound': -0.3767}", country='USA'),
 Row(_c0=3, geo=None, text=' Get ready for mass event crowd cancellations across the World starting this weekend: cricket in #Australia in empty st…', user='InfectiousDz', locatio

In [30]:

x = spark.sparkContext.parallelize([1,4,8])
y = x.flatMap(lambda x: (x, x*x))
print(x.collect())
print(y.collect())

[1, 4, 8]
[1, 1, 4, 16, 8, 64]


In [31]:
x = spark.sparkContext.parallelize([1,4,8])
y = x.map(lambda x: (x, x*x))
print(x.collect())
print(y.collect())

[1, 4, 8]
[(1, 1), (4, 16), (8, 64)]


In [32]:
x

ParallelCollectionRDD[62] at readRDDFromFile at PythonRDD.scala:247

In [33]:
twitter_df.select("text").show()

+--------------------+
|                text|
+--------------------+
|What is God sayin...|
|"BREAKING: ""this...|
| #Coronavirus tes...|
| Get ready for ma...|
| The #coronavirus...|
| COVID-19 update ...|
| It’s painful to ...|
| 📽️Friends, I wi...|
| Questions about ...|
|How they’re deali...|
| BREAKING: Democr...|
| “If we close dow...|
| I pity the poor ...|
| We’re the heck w...|
| I don't think sh...|
| Well written, so...|
| 1/2 CDC Director...|
| In all seriousne...|
| Wash your hands....|
| #CoronaVirusCana...|
+--------------------+
only showing top 20 rows



In [34]:
twitter_df.select("text","user").show()

+--------------------+--------------------+
|                text|                user|
+--------------------+--------------------+
|What is God sayin...|          petodinice|
|"BREAKING: ""this...| but i took the t...|
| #Coronavirus tes...|              cek422|
| Get ready for ma...|        InfectiousDz|
| The #coronavirus...|          vic_gibson|
| COVID-19 update ...|      StewartNgilana|
| It’s painful to ...|           BWheatnyc|
| 📽️Friends, I wi...|             LorseaR|
| Questions about ...|         straightj23|
|How they’re deali...|       _______coolio|
| BREAKING: Democr...|      champagneaylin|
| “If we close dow...|       YorkLawLondon|
| I pity the poor ...|      BeesonMargaret|
| We’re the heck w...|      harrytiffanyiv|
| I don't think sh...|         grammyheath|
| Well written, so...|      barbara_ellena|
| 1/2 CDC Director...|               fatal|
| In all seriousne...|          pwjkmiller|
| Wash your hands....|        Mrrandy123RP|
| #CoronaVirusCana...|           

In [35]:
twitter_df.rdd.map(lambda line: line.text.split(" ")).take(5)


[['What', 'is', 'God', 'saying', 'to', 'us', 'about', '#coronavirus', '?'],
 ['"BREAKING:', '""this', 'is', 'disappointing'],
 ['',
  '#Coronavirus',
  'testing',
  'must',
  'be',
  'made',
  'free',
  'to',
  'the',
  'public',
  'if',
  'we',
  'are',
  'going',
  'to',
  'understand',
  'the',
  'scope',
  'of',
  'this',
  'crisis.',
  'Anything',
  'le…'],
 ['',
  'Get',
  'ready',
  'for',
  'mass',
  'event',
  'crowd',
  'cancellations',
  'across',
  'the',
  'World',
  'starting',
  'this',
  'weekend:',
  'cricket',
  'in',
  '#Australia',
  'in',
  'empty',
  'st…'],
 ['',
  'The',
  '#coronavirus',
  'pandemic',
  'is',
  'revealing',
  'just',
  'how',
  'closely',
  'we',
  'are',
  'all',
  'bound',
  'together...[A',
  'thread]',
  '']]

In [36]:
twitter_df.rdd.flatMap(lambda line: line.text.split(" ")).take(100)


['What',
 'is',
 'God',
 'saying',
 'to',
 'us',
 'about',
 '#coronavirus',
 '?',
 '"BREAKING:',
 '""this',
 'is',
 'disappointing',
 '',
 '#Coronavirus',
 'testing',
 'must',
 'be',
 'made',
 'free',
 'to',
 'the',
 'public',
 'if',
 'we',
 'are',
 'going',
 'to',
 'understand',
 'the',
 'scope',
 'of',
 'this',
 'crisis.',
 'Anything',
 'le…',
 '',
 'Get',
 'ready',
 'for',
 'mass',
 'event',
 'crowd',
 'cancellations',
 'across',
 'the',
 'World',
 'starting',
 'this',
 'weekend:',
 'cricket',
 'in',
 '#Australia',
 'in',
 'empty',
 'st…',
 '',
 'The',
 '#coronavirus',
 'pandemic',
 'is',
 'revealing',
 'just',
 'how',
 'closely',
 'we',
 'are',
 'all',
 'bound',
 'together...[A',
 'thread]',
 '',
 '',
 'COVID-19',
 'update',
 'as',
 'of',
 'this',
 'morning:1.',
 'Death',
 'toll',
 'in',
 'Italy',
 'passes',
 '1,0002.',
 "Arsenal's",
 'head',
 'coach',
 'Arteta',
 'tests',
 'positive3.',
 'US…',
 '',
 'It’s',
 'painful',
 'to',
 'say,',
 'but',
 'as',
 'an']

In [37]:
corona_df.show()


+---+----------------+--------------------+--------+---------+----------+---------+-----+---------+--------------------+----+
|_c0|           State|             Country|     Lat|     Long|      Date|Confirmed|Death|Recovered|       state_cleaned|City|
+---+----------------+--------------------+--------+---------+----------+---------+-----+---------+--------------------+----+
|  0|            null|            Thailand|    15.0|    101.0|2020-01-22|        2|    0|        0|             Bangkok|null|
|  1|            null|               Japan|    36.0|    138.0|2020-01-22|        2|    0|        0|             Hiraide|null|
|  2|            null|           Singapore|  1.2833| 103.8333|2020-01-22|        0|    0|        0|           Singapore|null|
|  3|            null|               Nepal| 28.1667|    84.25|2020-01-22|        0|    0|        0|           Kathmandu|null|
|  4|            null|            Malaysia|     2.5|    112.5|2020-01-22|        0|    0|        0|             Sarawa

In [38]:
corona_df.filter("Country='US'").show()


+---+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|_c0|           State|Country|               Lat|     Long|      Date|Confirmed|Death|Recovered|   state_cleaned|            City|
+---+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
| 98|      Washington|     US|           47.4009|-121.4905|2020-01-22|        0|    0|        0|      Washington|      Washington|
| 99|        New York|     US|           42.1657| -74.9481|2020-01-22|        0|    0|        0|        New York|        New York|
|100|      California|     US|           36.1162|-119.6816|2020-01-22|        0|    0|        0|      California|      California|
|101|   Massachusetts|     US|           42.2302| -71.5301|2020-01-22|        0|    0|        0|   Massachusetts|   Massachusetts|
|102|Diamond Princess|     US|           35.4437|  139.638|2020-01-22|        0|   

In [39]:
corona_df.filter("Country='US'").sort(col("Date"), ascending=False).show()


+-----+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|  _c0|           State|Country|               Lat|     Long|      Date|Confirmed|Death|Recovered|   state_cleaned|            City|
+-----+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|27764|      Washington|     US|           47.4009|-121.4905|2020-03-20|     1524|   83|        0|      Washington|      Washington|
|27784|         Arizona|     US|           33.7298|-111.4312|2020-03-20|       78|    0|        0|         Arizona|         Arizona|
|27765|        New York|     US|           42.1657| -74.9481|2020-03-20|     8310|   42|        0|        New York|        New York|
|27766|      California|     US|           36.1162|-119.6816|2020-03-20|     1177|   23|        0|      California|      California|
|27767|   Massachusetts|     US|           42.2302| -71.5301|2020-03-

In [40]:
corona_df.filter("Country='US'").orderBy(col("Date"), ascending=False).show()


+-----+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|  _c0|           State|Country|               Lat|     Long|      Date|Confirmed|Death|Recovered|   state_cleaned|            City|
+-----+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|27764|      Washington|     US|           47.4009|-121.4905|2020-03-20|     1524|   83|        0|      Washington|      Washington|
|27784|         Arizona|     US|           33.7298|-111.4312|2020-03-20|       78|    0|        0|         Arizona|         Arizona|
|27765|        New York|     US|           42.1657| -74.9481|2020-03-20|     8310|   42|        0|        New York|        New York|
|27766|      California|     US|           36.1162|-119.6816|2020-03-20|     1177|   23|        0|      California|      California|
|27767|   Massachusetts|     US|           42.2302| -71.5301|2020-03-

In [41]:
corona_df.filter("Country='US'").sortWithinPartitions([col("Date"),col("Confirmed")], ascending=False).show()


+-----+--------------+-------+-------+---------+----------+---------+-----+---------+--------------+--------------+
|  _c0|         State|Country|    Lat|     Long|      Date|Confirmed|Death|Recovered| state_cleaned|          City|
+-----+--------------+-------+-------+---------+----------+---------+-----+---------+--------------+--------------+
|27765|      New York|     US|42.1657| -74.9481|2020-03-20|     8310|   42|        0|      New York|      New York|
|27764|    Washington|     US|47.4009|-121.4905|2020-03-20|     1524|   83|        0|    Washington|    Washington|
|27766|    California|     US|36.1162|-119.6816|2020-03-20|     1177|   23|        0|    California|    California|
|27773|    New Jersey|     US|40.2989|  -74.521|2020-03-20|      890|   11|        0|    New Jersey|    New Jersey|
|27776|      Illinois|     US|40.3495| -88.9861|2020-03-20|      585|    5|        0|      Illinois|      Illinois|
|27772|       Florida|     US|27.7663| -81.6868|2020-03-20|      563|   

In [42]:
corona_df.describe().show()


+-------+-----------------+---------+-----------+------------------+------------------+----------+------------------+------------------+------------------+-------------+-----------+
|summary|              _c0|    State|    Country|               Lat|              Long|      Date|         Confirmed|             Death|         Recovered|state_cleaned|       City|
+-------+-----------------+---------+-----------+------------------+------------------+----------+------------------+------------------+------------------+-------------+-----------+
|  count|            28143|    19116|      28143|             28143|             28143|     28143|             28143|             28143|             28143|        28143|      14573|
|   mean|          14071.0|     null|       null|30.965553459118834|-34.57031257861667|      null|161.88245744945456| 5.494368048893153| 60.17290267562094|         null|       null|
| stddev|8124.328649186959|     null|       null|19.365472826597646| 80.78375872452575|   

In [43]:
corona_df.printSchema()


root
 |-- _c0: integer (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Death: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- state_cleaned: string (nullable = true)
 |-- City: string (nullable = true)



In [44]:
corona_df.filter('Confirmed>10000').sort(col('Confirmed')).show()


+-----+------+-------+-------+--------+----------+---------+-----+---------+-------------+----+
|  _c0| State|Country|    Lat|    Long|      Date|Confirmed|Death|Recovered|state_cleaned|City|
+-----+------+-------+-------+--------+----------+---------+-----+---------+-------------+----+
|24005|  null|   Iran|   32.0|    53.0|2020-03-12|    10075|  429|     2959|       Tehran|null|
|22912|  null|  Italy|   43.0|    12.0|2020-03-10|    10149|  631|      724|         Rome|null|
|27346|France| France|46.2276|  2.2137|2020-03-19|    10871|  243|       12|       France|null|
| 5401| Hubei|  China|30.9756|112.2707|2020-02-02|    11177|  350|      295|        Hubei|null|
|24482|  null|   Iran|   32.0|    53.0|2020-03-13|    11364|  514|     2959|       Tehran|null|
|26253|  null|  Spain|   40.0|    -4.0|2020-03-17|    11748|  533|     1028|       Toledo|null|
|26723|  null|Germany|   51.0|     9.0|2020-03-18|    12327|   28|      105|       Berlin|null|
|23866|  null|  Italy|   43.0|    12.0|2

In [45]:
corona_df.filter('Confirmed>10000').approxQuantile('Confirmed',[0.25, 0.5, 0.75, 0.9, 0.95],0.9)


[10075.0, 10075.0, 10075.0, 10075.0, 67800.0]

In [46]:
corona_df.agg({"Date":"max"}).collect()


[Row(max(Date)='2020-03-20')]

In [47]:
corona_df.agg({"Date":"max","confirmed":"max"}).collect()


[Row(max(confirmed)=67800, max(Date)='2020-03-20')]

In [0]:
max_date=corona_df.agg({"Date":"max"})


In [50]:
max_date.show()


+----------+
| max(Date)|
+----------+
|2020-03-20|
+----------+



In [51]:

import pyspark.sql.functions as F
corona_df.groupBy("Country","State_cleaned").agg(F.max("Date")).show()

+-------------------+--------------------+----------+
|            Country|       State_cleaned| max(Date)|
+-------------------+--------------------+----------+
|        Afghanistan|             Baghran|2020-03-20|
|            Albania|                E852|2020-03-20|
|            Algeria|            Timokten|2020-03-20|
|            Andorra|    Andorra la Vella|2020-03-20|
|             Angola|             Sautari|2020-03-20|
|Antigua and Barbuda|          St. John's|2020-03-20|
|          Argentina|        Buenos Aires|2020-03-20|
|            Armenia|             Yerevan|2020-03-20|
|          Australia|Australian Capita...|2020-03-20|
|          Australia|From Diamond Prin...|2020-03-20|
|          Australia|     New South Wales|2020-03-20|
|          Australia|  Northern Territory|2020-03-20|
|          Australia|          Queensland|2020-03-20|
|          Australia|     South Australia|2020-03-20|
|          Australia|            Tasmania|2020-03-20|
|          Australia|       

In [52]:
corona_df.join(corona_df.groupBy("Country","State_cleaned").agg(F.max("Date").alias("Date")),on=['Country', 'State_cleaned','Date'],how="inner").show()

+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|             Country|       state_cleaned|      Date|  _c0|           State|     Lat|     Long|Confirmed|Death|Recovered|City|
+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|            Thailand|             Bangkok|2020-03-20|27666|            null|    15.0|    101.0|      322|    1|       42|null|
|               Japan|             Hiraide|2020-03-20|27667|            null|    36.0|    138.0|      963|   33|      191|null|
|           Singapore|           Singapore|2020-03-20|27668|            null|  1.2833| 103.8333|      385|    0|      124|null|
|               Nepal|           Kathmandu|2020-03-20|27669|            null| 28.1667|    84.25|        1|    0|        1|null|
|            Malaysia|             Sarawak|2020-03-20|27670|            null|     2.5|    112.5|     103

In [53]:
corona_df.join(corona_df.groupBy("Country","State_cleaned").agg(F.max("Date").alias("Date")),on=['Country', 'State_cleaned','Date'],how="inner").sort("Confirmed",ascending=False).show(10)


+--------------+----------------+----------+-----+--------------+-------+-------------------+---------+-----+---------+--------+
|       Country|   state_cleaned|      Date|  _c0|         State|    Lat|               Long|Confirmed|Death|Recovered|    City|
+--------------+----------------+----------+-----+--------------+-------+-------------------+---------+-----+---------+--------+
|         China|           Hubei|2020-03-20|27820|         Hubei|30.9756|           112.2707|    67800| 3133|    58382|    null|
|         Italy|            Rome|2020-03-20|27682|          null|   43.0|               12.0|    47021| 4032|     4440|    null|
|         Spain|          Toledo|2020-03-20|27684|          null|   40.0|               -4.0|    20410| 1043|     1588|    null|
|       Germany|          Berlin|2020-03-20|27677|          null|   51.0|                9.0|    19848|   67|      180|    null|
|          Iran|          Tehran|2020-03-20|27821|          null|   32.0|               53.0|    

In [54]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

ws=Window().partitionBy("Country","State_cleaned").orderBy(col("Date").desc())

corona_df.withColumn("row_num", row_number().over(ws)).show()

+-----+-----+-----------+----+----+----------+---------+-----+---------+-------------+----+-------+
|  _c0|State|    Country| Lat|Long|      Date|Confirmed|Death|Recovered|state_cleaned|City|row_num|
+-----+-----+-----------+----+----+----------+---------+-----+---------+-------------+----+-------+
|27692| null|Afghanistan|33.0|65.0|2020-03-20|       24|    0|        1|      Baghran|null|      1|
|27215| null|Afghanistan|33.0|65.0|2020-03-19|       22|    0|        1|      Baghran|null|      2|
|26738| null|Afghanistan|33.0|65.0|2020-03-18|       22|    0|        1|      Baghran|null|      3|
|26261| null|Afghanistan|33.0|65.0|2020-03-17|       22|    0|        1|      Baghran|null|      4|
|25784| null|Afghanistan|33.0|65.0|2020-03-16|       21|    0|        1|      Baghran|null|      5|
|25307| null|Afghanistan|33.0|65.0|2020-03-15|       16|    0|        0|      Baghran|null|      6|
|24830| null|Afghanistan|33.0|65.0|2020-03-14|       11|    0|        0|      Baghran|null|      7|


In [55]:
corona_df.withColumn("row_num", row_number().over(ws)).where(col("row_num")==1).show()


+-----+--------------------+-------------------+--------+--------+----------+---------+-----+---------+--------------------+----+-------+
|  _c0|               State|            Country|     Lat|    Long|      Date|Confirmed|Death|Recovered|       state_cleaned|City|row_num|
+-----+--------------------+-------------------+--------+--------+----------+---------+-----+---------+--------------------+----+-------+
|27692|                null|        Afghanistan|    33.0|    65.0|2020-03-20|       24|    0|        1|             Baghran|null|      1|
|27761|                null|            Albania| 41.1533| 20.1683|2020-03-20|       70|    2|        0|                E852|null|      1|
|27695|                null|            Algeria| 28.0339|  1.6596|2020-03-20|       90|   11|       32|            Timokten|null|      1|
|27726|                null|            Andorra| 42.5063|  1.5218|2020-03-20|       75|    0|        1|    Andorra la Vella|null|      1|
|28136|                null|      

In [0]:
corona_max_df=corona_df.join(corona_df.groupBy("Country","State_cleaned").agg(F.max("Date").alias("Date")),on=['Country', 'State_cleaned','Date'],how="inner")

In [57]:
corona_max_df.show()


+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|             Country|       state_cleaned|      Date|  _c0|           State|     Lat|     Long|Confirmed|Death|Recovered|City|
+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|            Thailand|             Bangkok|2020-03-20|27666|            null|    15.0|    101.0|      322|    1|       42|null|
|               Japan|             Hiraide|2020-03-20|27667|            null|    36.0|    138.0|      963|   33|      191|null|
|           Singapore|           Singapore|2020-03-20|27668|            null|  1.2833| 103.8333|      385|    0|      124|null|
|               Nepal|           Kathmandu|2020-03-20|27669|            null| 28.1667|    84.25|        1|    0|        1|null|
|            Malaysia|             Sarawak|2020-03-20|27670|            null|     2.5|    112.5|     103

In [58]:
corona_df.groupby("Country").pivot("Date").agg(F.sum("Confirmed")).show()

+--------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|             Country|2020-01-22|2020-01-23|2020-01-24|2020-01-25|2020-01-26|2020-01-27|2020-01-28|2020-01-29|2020-01-30|2020-01-31|2020-02-01|2020-02-02|2020-02-03|2020-02-04|2020-02-05|2020-02-06|2020-02-07|2020-02-08|2020-02-09|2020-02-10|2020-02-11|2020-02-12|2020-02-13|2020-02-14|2020-02-15|2020-02-16|2020-02-17|2020-02-1

In [59]:
corona_df.filter("Country=='US'").crosstab("State","Date").show()


+-------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|         State_Date|2020-01-22|2020-01-23|2020-01-24|2020-01-25|2020-01-26|2020-01-27|2020-01-28|2020-01-29|2020-01-30|2020-01-31|2020-02-01|2020-02-02|2020-02-03|2020-02-04|2020-02-05|2020-02-06|2020-02-07|2020-02-08|2020-02-09|2020-02-10|2020-02-11|2020-02-12|2020-02-13|2020-02-14|2020-02-15|2020-02-16|2020-02-17|2020-02-18|

In [60]:
corona_max_df.groupBy("Country").agg({'Confirmed':'sum'}).show()


+--------------------+--------------+
|             Country|sum(Confirmed)|
+--------------------+--------------+
|            Thailand|           322|
|               Japan|           963|
|           Singapore|           385|
|               Nepal|             1|
|            Malaysia|          1030|
|              Canada|           943|
|           Australia|           791|
|            Cambodia|            51|
|           Sri Lanka|            73|
|             Germany|         19848|
|             Finland|           450|
|United Arab Emirates|           140|
|         Philippines|           230|
|               India|           244|
|               Italy|         47021|
|              Sweden|          1639|
|               Spain|         20410|
|             Belgium|          2257|
|               Egypt|           285|
|             Lebanon|           163|
+--------------------+--------------+
only showing top 20 rows



In [61]:

corona_max_df.groupBy("Country").agg({'Confirmed':'sum','Recovered':'sum','Death':'sum'}).orderBy("sum(Confirmed)", ascending=False).show()

+--------------+--------------+----------+--------------+
|       Country|sum(Recovered)|sum(Death)|sum(Confirmed)|
+--------------+--------------+----------+--------------+
|         China|         71266|      3253|         81250|
|         Italy|          4440|      4032|         47021|
|         Spain|          1588|      1043|         20410|
|       Germany|           180|        67|         19848|
|          Iran|          6745|      1433|         19644|
|            US|             0|       244|         19100|
|        France|            12|       450|         12726|
|  Korea, South|          1540|        94|          8652|
|   Switzerland|            15|        54|          5294|
|United Kingdom|            67|       178|          4014|
|   Netherlands|             2|       107|          3003|
|       Austria|             9|         6|          2388|
|       Belgium|             1|        37|          2257|
|        Norway|             1|         7|          1914|
|        Swede

In [62]:
corona_df.filter("Country=='Italy'").sort("Date",ascending=False).show()


+-----+-----+-------+----+----+----------+---------+-----+---------+-------------+----+
|  _c0|State|Country| Lat|Long|      Date|Confirmed|Death|Recovered|state_cleaned|City|
+-----+-----+-------+----+----+----------+---------+-----+---------+-------------+----+
|27682| null|  Italy|43.0|12.0|2020-03-20|    47021| 4032|     4440|         Rome|null|
|27205| null|  Italy|43.0|12.0|2020-03-19|    41035| 3405|     4440|         Rome|null|
|26728| null|  Italy|43.0|12.0|2020-03-18|    35713| 2978|     4025|         Rome|null|
|26251| null|  Italy|43.0|12.0|2020-03-17|    31506| 2503|     2941|         Rome|null|
|25774| null|  Italy|43.0|12.0|2020-03-16|    27980| 2158|     2749|         Rome|null|
|25297| null|  Italy|43.0|12.0|2020-03-15|    24747| 1809|     2335|         Rome|null|
|24820| null|  Italy|43.0|12.0|2020-03-14|    21157| 1441|     1966|         Rome|null|
|24343| null|  Italy|43.0|12.0|2020-03-13|    17660| 1266|     1439|         Rome|null|
|23866| null|  Italy|43.0|12.0|2

In [63]:
corona_max_df.withColumn("Active",corona_max_df.Confirmed - corona_max_df.Recovered - corona_max_df.Death).sort("Active",ascending=False).show()

+--------------+----------------+----------+-----+--------------+-------+-------------------+---------+-----+---------+----------+------+
|       Country|   state_cleaned|      Date|  _c0|         State|    Lat|               Long|Confirmed|Death|Recovered|      City|Active|
+--------------+----------------+----------+-----+--------------+-------+-------------------+---------+-----+---------+----------+------+
|         Italy|            Rome|2020-03-20|27682|          null|   43.0|               12.0|    47021| 4032|     4440|      null| 38549|
|       Germany|          Berlin|2020-03-20|27677|          null|   51.0|                9.0|    19848|   67|      180|      null| 19601|
|         Spain|          Toledo|2020-03-20|27684|          null|   40.0|               -4.0|    20410| 1043|     1588|      null| 17779|
|        France|          France|2020-03-20|27823|        France|46.2276|             2.2137|    12612|  450|       12|      null| 12150|
|          Iran|          Tehran|2

In [0]:
corona_max_df=corona_max_df.withColumn("Active",corona_max_df.Confirmed - corona_max_df.Recovered - corona_max_df.Death)

In [65]:
corona_max_df.show(10)


+---------+----------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+------+
|  Country|   state_cleaned|      Date|  _c0|           State|     Lat|     Long|Confirmed|Death|Recovered|City|Active|
+---------+----------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+------+
| Thailand|         Bangkok|2020-03-20|27666|            null|    15.0|    101.0|      322|    1|       42|null|   279|
|    Japan|         Hiraide|2020-03-20|27667|            null|    36.0|    138.0|      963|   33|      191|null|   739|
|Singapore|       Singapore|2020-03-20|27668|            null|  1.2833| 103.8333|      385|    0|      124|null|   261|
|    Nepal|       Kathmandu|2020-03-20|27669|            null| 28.1667|    84.25|        1|    0|        1|null|     0|
| Malaysia|         Sarawak|2020-03-20|27670|            null|     2.5|    112.5|     1030|    3|       87|null|   940|
|   Canada|British Columbia|2020-03-20|2

In [66]:
corona_max_df.groupBy("Country").sum("Active").orderBy("sum(Active)",ascending=False).show()


+--------------+-----------+
|       Country|sum(Active)|
+--------------+-----------+
|         Italy|      38549|
|       Germany|      19601|
|            US|      18856|
|         Spain|      17779|
|        France|      12264|
|          Iran|      11466|
|  Korea, South|       7018|
|         China|       6731|
|   Switzerland|       5225|
|United Kingdom|       3769|
|   Netherlands|       2894|
|       Austria|       2373|
|       Belgium|       2219|
|        Norway|       1906|
|        Sweden|       1607|
|       Denmark|       1327|
|      Portugal|       1009|
|      Malaysia|        940|
|        Canada|        922|
|       Czechia|        829|
+--------------+-----------+
only showing top 20 rows



In [67]:
corona_max_df.select("Country","State_cleaned","Confirmed","Recovered").filter(col("Country").isin('Australia','China')).cube("Country").sum().sort(asc("Country")).show(100)

+---------+--------------+--------------+
|  Country|sum(Confirmed)|sum(Recovered)|
+---------+--------------+--------------+
|     null|         82041|         71292|
|Australia|           791|            26|
|    China|         81250|         71266|
+---------+--------------+--------------+



In [68]:
corona_max_df.select("Country","State_cleaned","Confirmed","Recovered").filter(col("Country").isin('Australia','Canada')).cube("Country","State_cleaned").sum().sort(asc("Country")).show(100)

+---------+--------------------+--------------+--------------+
|  Country|       State_cleaned|sum(Confirmed)|sum(Recovered)|
+---------+--------------------+--------------+--------------+
|     null|  Northern Territory|             3|             0|
|     null|            Manitoba|            17|             0|
|     null|      Grand Princess|            10|             0|
|     null|   Western Australia|            64|             0|
|     null|            Tasmania|            10|             3|
|     null|        Saskatchewan|            20|             0|
|     null|    British Columbia|           271|             4|
|     null|         Nova Scotia|            15|             0|
|     null|From Diamond Prin...|             0|             0|
|     null|             Ontario|           308|             5|
|     null|Newfoundland and ...|             4|             0|
|     null|             Alberta|           146|             0|
|     null|          Queensland|           184|        

In [69]:
corona_max_df.select("Country","State_cleaned","Confirmed","Recovered").filter(col("Country").isin('Australia','Canada')).rollup("Country","State_cleaned").sum().sort(asc("Country")).show(100)

+---------+--------------------+--------------+--------------+
|  Country|       State_cleaned|sum(Confirmed)|sum(Recovered)|
+---------+--------------------+--------------+--------------+
|     null|                null|          1734|            35|
|Australia|            Tasmania|            10|             3|
|Australia|Australian Capita...|             6|             0|
|Australia|          Queensland|           184|             8|
|Australia|   Western Australia|            64|             0|
|Australia|                null|           791|            26|
|Australia|  Northern Territory|             3|             0|
|Australia|From Diamond Prin...|             0|             0|
|Australia|            Victoria|           121|             8|
|Australia|     South Australia|            50|             3|
|Australia|     New South Wales|           353|             4|
|   Canada|       New Brunswick|            11|             0|
|   Canada|        Saskatchewan|            20|        

In [70]:

corona_max_df.corr('Confirmed','Recovered')

0.8052934236742176

In [71]:
corona_max_df.cache()


DataFrame[Country: string, state_cleaned: string, Date: string, _c0: int, State: string, Lat: double, Long: double, Confirmed: int, Death: int, Recovered: int, City: string, Active: int]

In [72]:
from pyspark import StorageLevel
corona_max_df.persist(StorageLevel.MEMORY_AND_DISK)

DataFrame[Country: string, state_cleaned: string, Date: string, _c0: int, State: string, Lat: double, Long: double, Confirmed: int, Death: int, Recovered: int, City: string, Active: int]

In [75]:
%timeit corona_max_df.count()


10 loops, best of 3: 44.6 ms per loop


In [0]:
pd=corona_df.toPandas()


In [77]:
pd.corr()


,_c0,Lat,Long,Confirmed,Death,Recovered
_c0,1.000000,-0.001594,-0.005303,0.046789,0.043155,0.042857
Lat,-0.001594,1.000000,-0.383389,0.008531,0.007041,0.000778
Long,-0.005303,-0.383389,1.000000,0.098893,0.074918,0.079047
Confirmed,0.046789,0.008531,0.098893,1.000000,0.963376,0.853924
Death,0.043155,0.007041,0.074918,0.963376,1.000000,0.857292
Recovered,0.042857,0.000778,0.079047,0.853924,0.857292,1.000000


In [0]:
corona_max_df.createOrReplaceTempView("corona")


In [80]:
spark.sql("select * from corona").show()


+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+------+
|             Country|       state_cleaned|      Date|  _c0|           State|     Lat|     Long|Confirmed|Death|Recovered|City|Active|
+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+------+
|            Thailand|             Bangkok|2020-03-20|27666|            null|    15.0|    101.0|      322|    1|       42|null|   279|
|               Japan|             Hiraide|2020-03-20|27667|            null|    36.0|    138.0|      963|   33|      191|null|   739|
|           Singapore|           Singapore|2020-03-20|27668|            null|  1.2833| 103.8333|      385|    0|      124|null|   261|
|               Nepal|           Kathmandu|2020-03-20|27669|            null| 28.1667|    84.25|        1|    0|        1|null|     0|
|            Malaysia|             Sarawak|2020-03-20|2